In [2]:
import json

import pandas as pd
import requests
from calitp_data_analysis.sql import to_snakecase

In [3]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

## Catastrophic

In [4]:
expired_df = pd.read_excel("./feed_info_expired_feeds_2023-09-21T19_13_29.263825Z.xlsx")

/opt/conda/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [6]:
expired_df = to_snakecase(expired_df)

In [7]:
# Don't look at LACMTA stuff
no_lacmta = expired_df[~expired_df["dim_gtfs_datasets_→_uri"].str.contains("LACMTA")]

In [38]:
no_lacmta2 = no_lacmta[
    [
        "dim_provider_gtfs_data_→_service_name",
        "dim_county_geography_→_caltrans_district",
        "date",
    ]
].sort_values(
    ["dim_provider_gtfs_data_→_service_name", "date"], ascending=[False, False]
)

In [39]:
open_airtable_tickets = to_snakecase(
    pd.read_csv("./Transit Data Quality Issues-Open Issues by Distrcit (1).csv")
)

In [40]:
open_airtable_tickets = open_airtable_tickets.fillna("None")

In [41]:
open_airtable_tickets = (
    open_airtable_tickets[
        [
            "description",
            "gtfs_datasets",
            "services",
        ]
    ]
    .sort_values(["gtfs_datasets", "services"])
    .reset_index(drop=True)
)

In [42]:
open_airtable_tickets["airtable_ticket"] = "yes"

In [43]:
no_lacmta2_summary = (
    no_lacmta2.groupby(["dim_provider_gtfs_data_→_service_name"])
    .agg({"date": "count"})
    .reset_index()
    .rename(columns={"date": "# of days with expired feed"})
)

In [44]:
pd.merge(
    no_lacmta2_summary,
    open_airtable_tickets,
    left_on=["dim_provider_gtfs_data_→_service_name"],
    right_on=["services"],
    how="left",
)

,dim_provider_gtfs_data_→_service_name,# of days with expired feed,gtfs_datasets,services,airtable_ticket
0,Morro Bay Transit,14,Morro Bay Schedule,Morro Bay Transit,yes
1,Plumas Transit Systems,14,Plumas Schedule,Plumas Transit Systems,yes
2,Sage Stage Intercity,2,NaN,NaN,NaN
3,TRACER,14,Tracy Schedule,TRACER,yes


In [45]:
no_lacmta2

,dim_provider_gtfs_data_→_service_name,dim_county_geography_→_caltrans_district,date
8,TRACER,10,2023-09-20
18,TRACER,10,2023-09-19
28,TRACER,10,2023-09-18
38,TRACER,10,2023-09-17
48,TRACER,10,2023-09-16
58,TRACER,10,2023-09-15
68,TRACER,10,2023-09-14
78,TRACER,10,2023-09-13
88,TRACER,10,2023-09-12
98,TRACER,10,2023-09-11


## Incompleteness

In [46]:
trip_update_df = to_snakecase(
    pd.read_excel(
        "./gtfs_rt_trip_updates_completeness__last_14_days__2023-09-25T18_11_34.892466Z.xlsx"
    )
)

/opt/conda/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [47]:
vp_df = to_snakecase(
    pd.read_excel(
        "gtfs_rt_vehicle_positions_completeness__last_14_days__2023-09-25T18_11_37.967895Z.xlsx"
    )
)

/opt/conda/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [48]:
low_tu = (
    trip_update_df[trip_update_df["%_of_trips_with_tu_messages"] < 41].sort_values(
        ["%_of_trips_with_tu_messages"]
    )
).reset_index(drop=True)

In [49]:
low_vp = (
    vp_df[vp_df["%_of_trips_with_vp_messages"] < 41].sort_values(
        ["%_of_trips_with_vp_messages"]
    )
).reset_index(drop=True)

In [50]:
incomplete = pd.merge(low_tu, low_vp, on="name", how="outer")
incomplete = incomplete.sort_values(["name"]).reset_index(drop=True)

In [51]:
incomplete = incomplete.fillna("OK")

In [62]:
# open_airtable_tickets

In [53]:
incomplete

,name,%_of_trips_with_tu_messages,%_of_trips_with_vp_messages
0,Anaheim Resort Schedule,21.11,17.73
1,Bay Area 511 ACE Schedule,0.00,0.00
2,Bay Area 511 Angel Island-Tiburon Ferry Schedule,0.00,0.00
3,Bay Area 511 BART Schedule,39.31,0.00
4,Bay Area 511 Capitol Corridor Schedule,0.00,0.00
5,Bay Area 511 Commute.org Schedule,0.00,0.00
6,Bay Area 511 Golden Gate Ferry Schedule,0.00,0.00
7,Bay Area 511 MVGO Schedule,0.00,0.00
8,Bay Area 511 Mission Bay Schedule,0.00,0.00
9,Bay Area 511 Rio Vista Delta Breeze Schedule,0.00,0.00


In [54]:
# incomplete.name = incomplete.name.str.replace('Schedule','')

In [67]:
# Read in 511
df_511 = to_snakecase(pd.read_excel("./API511.xlsx"))

In [80]:
df_511 = df_511.rename(columns={df_511.columns[0]: "new"})

In [81]:
df_511.columns

Index(['new'], dtype='object')

In [82]:
df_511_2 = df_511[df_511["new"].str.contains(("Name|Monitored"))].reset_index(drop=True)

In [91]:
df_511_2.new = (
    df_511_2.new.str.replace(">", "")
    .str.replace("ShortName", "")
    .str.replace("Name", "")
    .str.replace("Monitored", "")
    .str.replace(">", "")
    .str.replace("/", "")
)

In [103]:
df_511_2[(df_511_2.new == "true").shift(1).fillna(False)]

,new
13,AC TRANSIT
22,Bay Area Rapid Transit
25,Caltrain
37,County Connection
40,Dumbarton Express Consortium
43,Emery Go-Round
46,FAST
52,Golden Gate Transit
55,Livermore Amador Valley Transit Authority
58,Marin Transit


In [101]:
open_airtable_tickets

,gtfs_datasets,services,airtable_ticket
0,"AC Transit Alerts,Bay Area 511 Regional Alerts",AC Transit,yes
1,Alhambra Schedule,Alhambra Community Transit,yes
2,Anaheim Resort Schedule,Anaheim Resort Transportation,yes
3,Arcadia Schedule,Arcadia Transit,yes
4,Artesia Schedule,Artesia Transit,yes
5,Arvin Schedule,Arvin Transit,yes
6,Auburn Schedule,Auburn Transit,yes
7,Baldwin Park Schedule,Baldwin Park Express,yes
8,Bay Area 511 Regional VehiclePositions,AC Transit,yes
9,Bay Area 511 Union City Transit VehiclePositions,Union City Transit,yes


In [96]:
incomplete2 = pd.merge(
    incomplete,
    open_airtable_tickets,
    left_on="name",
    right_on="gtfs_datasets",
    how="left",
    indicator=True,
).sort_values("name")

In [97]:
incomplete2

,name,%_of_trips_with_tu_messages,%_of_trips_with_vp_messages,gtfs_datasets,services,airtable_ticket,_merge
0,Anaheim Resort Schedule,21.11,17.73,Anaheim Resort Schedule,Anaheim Resort Transportation,yes,both
1,Bay Area 511 ACE Schedule,0.00,0.00,NaN,NaN,NaN,left_only
2,Bay Area 511 Angel Island-Tiburon Ferry Schedule,0.00,0.00,NaN,NaN,NaN,left_only
3,Bay Area 511 BART Schedule,39.31,0.00,NaN,NaN,NaN,left_only
4,Bay Area 511 Capitol Corridor Schedule,0.00,0.00,NaN,NaN,NaN,left_only
5,Bay Area 511 Commute.org Schedule,0.00,0.00,NaN,NaN,NaN,left_only
6,Bay Area 511 Golden Gate Ferry Schedule,0.00,0.00,NaN,NaN,NaN,left_only
7,Bay Area 511 MVGO Schedule,0.00,0.00,NaN,NaN,NaN,left_only
8,Bay Area 511 Mission Bay Schedule,0.00,0.00,NaN,NaN,NaN,left_only
9,Bay Area 511 Rio Vista Delta Breeze Schedule,0.00,0.00,NaN,NaN,NaN,left_only
